A colouring of a graph G is an assignment of a colour to each vertex of $G$, so that no two adjacent vertices receive the same colour. The chromatic number of $G$, denoted by $\chi(G)$, is the smallest number of colours for which it is possible to produce a colouring. It is believed that finding the chromatic number of a graph $G$ is, in general, very hard.

The space $\mathcal{G}(n, p)$ is that of graphs with $n$ labelled vertices, edges appearing independently and at random with probability $p$.

The space $\mathcal{G}_k(n, p)$ differs from $\mathcal{G}(n, p)$ only in that $ij$ is never an edge if $i - j \equiv 0 \pmod{k}$.

The greedy algorithm colours a graph whose vertex set is ordered by colouring vertices one at a time in the order given, using colours from $\{1, 2, 3, \dots\}$. The colour chosen for a vertex is the least colour from among those not already assigned to any previously coloured neighbours.

In [1]:
import networkx as nx
import random

def generate_Gnp(n, p):
    '''
    Generates a random graph G(n, p).
    '''
    G = nx.Graph()
    G.add_nodes_from(range(n))
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < p:
                G.add_edge(i, j)
    return G

def generate_Gknp(k, n, p):
    '''
    Generates a random graph G_k(n, p).
    '''
    G = nx.Graph()
    G.add_nodes_from(range(n))
    for i in range(n):
        for j in range(i + 1, n):
            # Add an edge only if (j - i) is not a multiple of k
            if (j - i) % k != 0:
                if random.random() < p:
                    G.add_edge(i, j)
    return G

def greedy_colouring(graph, order):
    '''
    Applies the greedy colouring algorithm to a graph with a given vertex order.
    Returns the number of colours used.
    '''
    colours = {}
    # Vertices are coloured one by one in the given order
    for vertex in order:
        neighbor_colours = {colours.get(neighbor) for neighbor in graph.neighbors(vertex)}

        # Find the smallest colour not used by neighbors
        colour = 1
        while colour in neighbor_colours:
            colour += 1
        colours[vertex] = colour

    # The total number of colours is the maximum colour value assigned
    if not colours:
        return 0
    return max(colours.values())

We use the following order strategies:
1. By increasing degree.
2. By decreasing degree.
3. Where $v_j$ has minimum degree in the graph $G - \{v_{j+1}, \dots, v_n\}$.
4. At random.

In [2]:
def order_by_increasing_degree(graph):
    '''
    Orders vertices by their degree in increasing order.
    '''
    return sorted(graph.nodes(), key=lambda v: graph.degree(v))

def order_by_decreasing_degree(graph):
    '''
    Orders vertices by their degree in decreasing order.
    '''
    return sorted(graph.nodes(), key=lambda v: graph.degree(v), reverse=True)

def order_smallest_last(graph):
    '''
    Orders vertices using the smallest-last heuristic.
    The vertex with the minimum degree in the remaining graph is placed next in the order.
    '''
    g_copy = graph.copy()
    ordering = []
    while g_copy.number_of_nodes() > 0:
        # Find the vertex with the minimum degree in the current subgraph
        min_degree_vertex = min(g_copy.nodes(), key=lambda v: g_copy.degree(v))
        ordering.append(min_degree_vertex)
        g_copy.remove_node(min_degree_vertex)

    # The greedy algorithm colours from v1 to vn, so we reverse the removal order.
    return ordering[::-1]

def order_at_random(graph):
    '''
    Orders vertices randomly.
    '''
    nodes = list(graph.nodes())
    random.shuffle(nodes)
    return nodes


In [3]:
def run_experiment(graph_generator, params, description, num_of_graphs = 10):
    '''
    Runs the full experiment for a given graph type.
    - Generates n graphs.
    - Applies 4 ordering strategies to each.
    - Prints the number of colours used.
    '''
    ordering_strategies = {
        "Increasing Degree": order_by_increasing_degree,
        "Decreasing Degree": order_by_decreasing_degree,
        "Smallest Last": order_smallest_last,
        "Random": order_at_random
    }

    for i in range(10):
        print(f"\n--- Graph {i+1}/10 ---")
        G = graph_generator(*params)
        for name, func in ordering_strategies.items():
            vertex_order = func(G)
            num_colours = greedy_colouring(G, vertex_order)
            print(f"{name:<18} {num_colours} colours")

# Test on ten graphs in G(70, 0.5)
run_experiment(generate_Gnp, params=(70, 0.5), description="G(70, 0.5)", num_of_graphs = 10)
# Test on ten graphs in G_3(70, 0.75)
run_experiment(generate_Gknp, params=(3, 70, 0.75), description="G_3(70, 0.75)", num_of_graphs = 10)


--- Graph 1/10 ---
Increasing Degree  18 colours
Decreasing Degree  14 colours
Smallest Last      17 colours
Random             17 colours

--- Graph 2/10 ---
Increasing Degree  19 colours
Decreasing Degree  16 colours
Smallest Last      15 colours
Random             17 colours

--- Graph 3/10 ---
Increasing Degree  18 colours
Decreasing Degree  15 colours
Smallest Last      16 colours
Random             17 colours

--- Graph 4/10 ---
Increasing Degree  17 colours
Decreasing Degree  16 colours
Smallest Last      16 colours
Random             18 colours

--- Graph 5/10 ---
Increasing Degree  18 colours
Decreasing Degree  14 colours
Smallest Last      15 colours
Random             16 colours

--- Graph 6/10 ---
Increasing Degree  17 colours
Decreasing Degree  15 colours
Smallest Last      14 colours
Random             16 colours

--- Graph 7/10 ---
Increasing Degree  17 colours
Decreasing Degree  14 colours
Smallest Last      15 colours
Random             16 colours

--- Graph 8/10 ---


Decreasing degree and smallest last ordering strategies tend to use the fewest colours. These heuristics prioritize colouring the vertices with the most constraints first, which often leads to more efficient colour usage. Conversely, increasing degree is generally not a very effective strategy. Random ordering provides a baseline but is typically worse than the more structured heuristics like decreasing degree.

Guaranteed 3-Colouring for $\mathcal{G}_3(70, 0.75)$: By processing the vertices in their natural numerical order $(0, 1, 2, 3, \dots)$ we can ensure a 3-colouring.

When the greedy algorithm considers a vertex $v$, it assigns the smallest colour not used by its already coloured neighbours. In the natural ordering, the neighbours of $v$ will always have labels less than $v$.

Consider the possible colours for each vertex based on its label modulo $3$:
*   Vertices with label $\equiv 0 \pmod 3$: These vertices are not connected to each other, so they can all be assigned the same colour $1$.
*   Vertices with label $\equiv 1 \pmod 3$: Similarly, these vertices form an independent set and can all be assigned colour $2$.
*   Vertices with label $\equiv 2 \pmod 3$: Finally, these vertices can be assigned colour 3.

When colouring any vertex $v$, its neighbours can only have labels that are not congruent to v's label modulo $3$.  Since we have a distinct colour for each of these congruence classes, at most two distinct colours will have been used on its neighbours, leaving at least one of the first three colours available. The smallest last ordering also reproduces this in an isomorphic way.

The choice of $p = 0.75$ is significant because it creates a graph that is dense enough to be interesting. A high probability ensures that many of the permissible edges actually exist, making the graph non-trivial to colour without the insight of the special ordering.

In [4]:
def construct_worst_case_graph(n):
    '''
    Constructs the graph G of order 3n with χ(G)=3, but on which
    greedy might need n + 2 colours.

    The graph has 2n+2 core vertices and n-2 isolated vertices.
    '''
    if n < 2:
        raise ValueError("This construction is defined for n >= 2")

    G = nx.Graph()

    # Define vertex sets using clear labels
    a_nodes = [f'a_{i}' for i in range(1, n + 1)]
    b_nodes = [f'b_{i}' for i in range(1, n + 1)]
    c_nodes = ['c_1', 'c_2']
    # Add n-2 isolated vertices to make the total 3n
    iso_nodes = [f'iso_{i}' for i in range(1, n - 1)]

    G.add_nodes_from(a_nodes + b_nodes + c_nodes + iso_nodes)

    # Edges between a_i and b_j for all i != j
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            if i != j:
                G.add_edge(f'a_{i}', f'b_{j}')

    # c1 is connected to all 'a' vertices
    for a_node in a_nodes:
        G.add_edge('c_1', a_node)

    # c2 is connected to all 'b' vertices
    for b_node in b_nodes:
        G.add_edge('c_2', b_node)

    # Edge between c1 and c2
    G.add_edge('c_1', 'c_2')
    # Add an edge to create a triangle and make the graph 3-chromatic
    G.add_edge('a_1', 'a_2')

    return G

Let $G$ be the above constructed graph. Since the vertices $a_1, a_2, b_3$ are mutually connected, they form a $3$-cycle so the chromomatic number $\chi(G)$ is at least $3$. Conversely, three colours are sufficient because we can assign colour $1$ to $c_1$ and all the $b$ vertices; colour $2$ to $c_2$ and all the $a$ vertices except $a_1$; and finally colour $3$ to $a_1$.

Our greedy colouring takes $n+2$ colours when given the malicious ordering,
\begin{equation}
    a_1, b_1, a_2, b_2, ..., a_n, b_n, c_1, c_2.
\end{equation}
First, $a_1$ is assigned colour $1$. Then $b_1$ is also assigned colour $1$ as these initial vertices do not share an edge. Now $a_2$ is connected to $a_1$ and $b_1$ both colour 1, so it must take a new colour $2$.

In general, when the algorithm colours $a_k$, it is connected to $\{b_1, b_2, \dots, b_{k-1}\}$. By this point, these vertices have been assigned the colours $\{1, 2, \dots, k-1\}$ respectively. The algorithm introduces a new colour $k$ for $a_k$.
After this, it colours $b_k$, which is connected to $\{a_1, a_2, \dots, a_{k-1}\}$. These also have the colours $\{1, 2, \dots, k-1\}$ so $b_k$ is coloured $k$. After vertices $a_n$ and $b_n$, we have used a total of $n$ colours.

Finally, $c_1$ is connected to every vertex in set $A$ where the colours $\{1, 2, 3, \dots, n\}$ are all in use. Thus, $c_1$ must take a new colour $n+1$. Then $c_2$ is connected to every vertex in set $B$ and is also connected to $c_1$. Every single colour from $1$ to $n+1$ is used by its neighbors. Hence, a new colour $n+2$ is introduced to colour $c_2$.

In [5]:
n = 20
print(f"Verification for n = {n}")
print(f"Graph has 3n = {3*n} vertices.")
print(f"Optimal chromatic number χ(G) is 3.")
G = construct_worst_case_graph(n)

# Bad ordering.
worst_case_order = []
for i in range(1, n + 1):
    worst_case_order.append(f'a_{i}')
    worst_case_order.append(f'b_{i}')
worst_case_order.extend(['c_1', 'c_2'])
# Add isolated vertices at the end
worst_case_order.extend([f'iso_{i}' for i in range(1, n - 1)])

# Good ordering.
good_order = []
good_order.extend(f'a_{i}' for i in range(1, n + 1))
good_order.extend(f'b_{i}' for i in range(1, n + 1))
good_order.extend(['c_1', 'c_2'])
good_order.extend(f'iso_{i}' for i in range(1, n - 1))

# Run the greedy algorithm with the worst-case ordering
num_colours_worst = greedy_colouring(G, worst_case_order)

print(f"Testing with 'malicious' ordering: a_1, b_1, a_2, b_2,...")
print(f"  - Expected number of colours: {n+2}")
print(f"  - Actual number of colours used: {num_colours_worst}")

# Run the greedy algorithm with the "good" ordering
num_colours_good = greedy_colouring(G, good_order)

print(f"Testing with 'natural' ordering: a_1,..., a_n, b_1,..., b_n,...")
print(f"  - Expected number of colours: 3")
print(f"  - Actual number of colours used: {num_colours_good}")

Verification for n = 20
Graph has 3n = 60 vertices.
Optimal chromatic number χ(G) is 3.
Testing with 'malicious' ordering: a_1, b_1, a_2, b_2,...
  - Expected number of colours: 22
  - Actual number of colours used: 22
Testing with 'natural' ordering: a_1,..., a_n, b_1,..., b_n,...
  - Expected number of colours: 3
  - Actual number of colours used: 3
